In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
from Utils.writeSubmission import write_submission

In [2]:
dataset = pd.read_csv('../Input/interactions_and_impressions.csv')

C:\Users\Mike\AppData\Local\Temp\ipykernel_25048\1891551792.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Input/interactions_and_impressions.csv')


In [3]:
dataset = dataset.drop(columns=['Impressions'])
dataset

,UserID,ItemID,Data
0,0,11,1
1,0,21,0
2,0,21,0
3,0,21,0
4,0,21,1
...,...,...,...
5826501,41628,20448,0
5826502,41628,20896,1
5826503,41628,21506,1
5826504,41628,22882,0


In [4]:
dataset.dtypes

UserID    int64
ItemID    int64
Data      int64
dtype: object

In [5]:
userIDS = dataset['UserID'].unique()
userIDS

array([    0,     1,     2, ..., 41626, 41627, 41628], dtype=int64)

In [6]:
itemIDS = dataset['ItemID'].unique()
itemIDS

array([   11,    21,    22, ..., 20006, 20177,  2538], dtype=int64)

In [12]:
datasetCOO = sps.coo_matrix((dataset["Data"].values,
                          (dataset["UserID"].values, dataset["ItemID"].values)))
datasetCOO.data

array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [8]:
datasetCOO.row

array([    0,     0,     0, ..., 41628, 41628, 41628], dtype=int32)

In [9]:
datasetCOO.col

array([   11,    21,    21, ..., 21506, 22882, 23757], dtype=int32)

In [10]:
x = 0
URM = np.zeros((len(userIDS), 27968))
URM

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
while x < len(datasetCOO.data):
    if datasetCOO.data[x] == 0:
        URM[datasetCOO.row[x]][datasetCOO.col[x]] = int(5)
    elif datasetCOO.data[x] == 1 and URM[datasetCOO.row[x]][datasetCOO.col[x]] != int(5):
        URM[datasetCOO.row[x]][datasetCOO.col[x]] = int(1)
    x = x+1

In [14]:
URM = sps.csr_matrix(URM)

In [13]:
class TopPopRecommender(object):

    def fit(self, URM_train):

        self.URM_train = URM_train

        item_popularity = np.ediff1d(URM_train.tocsc().indptr)

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popular_items = np.argsort(item_popularity)
        self.popular_items = np.flip(self.popular_items, axis=0)

    def recommend(self, user_id, at=20, remove_seen=True):

        if remove_seen:
            seen_items = self.URM_train.indices[self.URM_train.indptr[user_id]:self.URM_train.indptr[user_id + 1]]

            unseen_items_mask = np.in1d(self.popular_items, seen_items, assume_unique=True, invert=True)

            unseen_items = self.popular_items[unseen_items_mask]

            recommended_items = unseen_items[0:at]

        else:
            recommended_items = self.popular_items[0:at]

        return recommended_items


In [14]:
topPopRecommender_removeSeen = TopPopRecommender()
topPopRecommender_removeSeen.fit(URM)

In [15]:
write_submission(recommender=topPopRecommender_removeSeen, target_users_path="../Input/data_target_users_test.csv",
                     out_path='../Output/{}_submission.csv'.format('TopPop'))